####  NOTES
>1. Verified that the model class and original model definition via repeated functio calls have extremely close validation scores **(0.2998 for normal and 0.3042 class for class)** ie, they are probably same.
>2. Added better code cleaning functionalities and mis-spelling corrections.


####  TODO
>1. There is text augmenter library https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb

# Colab

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# ! cp -r drive/My\ Drive/ml_projects/kaggle/google_quest/google-quest-challenge .

In [ ]:
# !ls  drive/My\ Drive/ml_projects/ml_datasets/nlp_pretrained/gensim-embeddings-dataset

In [ ]:
# ! cp drive/My\ Drive/ml_projects/ml_datasets/nlp_pretrained/gensim-embeddings-dataset/glove.twitter.27B.200d.gensim .
# ! cp drive/My\ Drive/ml_projects/ml_datasets/nlp_pretrained/gensim-embeddings-dataset/glove.twitter.27B.200d.gensim.vectors.npy .

# Start

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.run_this_cell {display: block !important;} </style"))

In [2]:
%tensorflow_version 2.x 

UsageError: Line magic function `%tensorflow_version` not found.


In [61]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, concatenate, LSTM, Bidirectional, Embedding, add, Dense
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

from tensorflow.keras import models, layers, optimizers


from datetime import datetime
from typing import List, Dict, Tuple, Any

from collections import Counter
from tqdm import trange, tqdm, tnrange, tqdm_notebook
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
tf.__version__

'2.1.0-rc0'

In [6]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Utility functions

In [7]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

In [8]:
def clean_text(x):

    x = str(x)
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^`{|}~' + '“”’':
        x = x.replace(punct, f' {punct}')
   
    for punct in '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—_':
        x = x.replace(punct, f' {punct}')
        
    for punct in puncts:
        x = x.replace(punct, f' {punct}')
 
    return x

In [9]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

In [10]:
mispell_dict = {"usepackage" : "use package",
                'instrumentsettingsid':'instrumental settings id',
                'RippleShaderProgram' : 'ripple shader program',
                'ShaderProgramConstants':'shader program constants',
                'storedElements':'stored elements',
                'stackSize' : 'stack size',
                '_':' ',
                "aren't" : "are not",
                "can't" : "cannot",
                "couldn't" : "could not",
                "couldnt" : "could not",
                "didn't" : "did not",
                "doesn't" : "does not",
                "doesnt" : "does not",
                "don't" : "do not",
                "hadn't" : "had not",
                "hasn't" : "has not",
                "haven't" : "have not",
                "havent" : "have not",
                "he'd" : "he would",
                "he'll" : "he will",
                "he's" : "he is",
                "i'd" : "I would",
                "i'd" : "I had",
                "i'll" : "I will",
                "i'm" : "I am",
                "isn't" : "is not",
                "it's" : "it is",
                "it'll":"it will",
                "i've" : "I have",
                "let's" : "let us",
                "mightn't" : "might not",
                "mustn't" : "must not",
                "shan't" : "shall not",
                "she'd" : "she would",
                "she'll" : "she will",
                "she's" : "she is",
                "shouldn't" : "should not",
                "shouldnt" : "should not",
                "that's" : "that is",
                "thats" : "that is",
                "there's" : "there is",
                "theres" : "there is",
                "they'd" : "they would",
                "they'll" : "they will",
                "they're" : "they are",
                "theyre":  "they are",
                "they've" : "they have",
                "we'd" : "we would",
                "we're" : "we are",
                "weren't" : "were not",
                "we've" : "we have",
                "what'll" : "what will",
                "what're" : "what are",
                "what's" : "what is",
                "what've" : "what have",
                "where's" : "where is",
                "who'd" : "who would",
                "who'll" : "who will",
                "who're" : "who are",
                "who's" : "who is",
                "who've" : "who have",
                "won't" : "will not",
                "wouldn't" : "would not",
                "you'd" : "you would",
                "you'll" : "you will",
                "you're" : "you are",
                "you've" : "you have",
                "'re": " are",
                "wasn't": "was not",
                "we'll":" will",
                "didn't": "did not",
                "tryin'":"trying"}

In [11]:
mispellings, mispellings_re = _get_mispell(mispell_dict)

In [12]:
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

### Loading Data

In [13]:
train_loc = '/kaggle/input/google-quest-challenge/train.csv'
test_loc= '/kaggle/input/google-quest-challenge/test.csv'

# train_loc = 'google-quest-challenge/train.csv'
# test_loc= 'google-quest-challenge/test.csv'

train_df = pd.read_csv(train_loc)
test_df = pd.read_csv(test_loc)


### Preprocessing

In [14]:
train_df["question_body"] = train_df["question_body"].apply(clean_text)
train_df["question_body"] = train_df["question_body"].apply(replace_typical_misspell)

train_df["question_title"] = train_df["question_title"].apply(clean_text)
train_df["question_title"] = train_df["question_title"].apply(replace_typical_misspell)

train_df["category"] = train_df["category"].apply(clean_text)
train_df["category"] = train_df["category"].apply(replace_typical_misspell)

train_df["answer"] = train_df["answer"].apply(clean_text)
train_df["answer"] = train_df["answer"].apply(replace_typical_misspell)


test_df["question_body"] = test_df["question_body"].apply(clean_text)
test_df["question_body"] = test_df["question_body"].apply(replace_typical_misspell)

test_df["question_title"] = test_df["question_title"].apply(clean_text)
test_df["question_title"] = test_df["question_title"].apply(replace_typical_misspell)

test_df["category"] = test_df["category"].apply(clean_text)
test_df["category"] = test_df["category"].apply(replace_typical_misspell)

test_df["answer"] = test_df["answer"].apply(clean_text)
test_df["answer"] = test_df["answer"].apply(replace_typical_misspell)

In [15]:
question_body = train_df['question_body']
answer = train_df['answer']
question_title = train_df["question_title"]
category = train_df["category"]

question_body_test = test_df['question_body']
answer_test = test_df['answer']
question_title_test = test_df["question_title"]
category_test = test_df["category"]

In [16]:
target = train_df[train_df.columns[-30:]]

### Creating Tokenziers

In [17]:
all_text = pd.concat([
                    train_df['question_body'],
                     train_df['answer'],
                     test_df['question_body'],
                     test_df['answer'],
                     train_df["question_title"],
                     train_df["category"],
                     test_df["question_title"],
                     test_df["category"]
                    ])

In [18]:
len(train_df), len(test_df), 6079*4 + 476 * 4, len(all_text)

(6079, 476, 26220, 26220)

In [19]:
tokenizer = Tokenizer(num_words=1000000, lower=False,filters='')

tokenizer.fit_on_texts(all_text)

In [20]:
len(tokenizer.word_counts), tokenizer.word_index['the'], tokenizer.word_index['ABC']

(100498, 4, 8783)

In [21]:
question_body = tokenizer.texts_to_sequences(question_body)
answer = tokenizer.texts_to_sequences(answer)
question_title = tokenizer.texts_to_sequences(question_title)
category = tokenizer.texts_to_sequences(category)

question_body_test = tokenizer.texts_to_sequences(question_body_test)
answer_test = tokenizer.texts_to_sequences(answer_test)
question_title_test = tokenizer.texts_to_sequences(question_title_test)
category_test = tokenizer.texts_to_sequences(category_test)

In [22]:
len(question_body), len(question_body[0])

(6079, 174)

In [23]:
vocab_size = len(tokenizer.word_index) + 1
maxlen = 245

In [24]:
question_body = pad_sequences(question_body, padding='post', maxlen=maxlen)
answer = pad_sequences(answer, padding='post', maxlen=maxlen)
question_title = pad_sequences(question_title, padding='post', maxlen=maxlen)
category = pad_sequences(category, padding='post', maxlen=maxlen)


question_body_test = pad_sequences(question_body_test, padding='post', maxlen=maxlen)
answer_test = pad_sequences(answer_test, padding='post', maxlen=maxlen)
question_title_test = pad_sequences(question_title_test, padding='post', maxlen=maxlen)
category_test = pad_sequences(category_test, padding='post', maxlen=maxlen)

### Build embedding matrix for the full dataset

Gensim word vectors can be found here https://www.kaggle.com/iezepov/gensim-embeddings-dataset

All UNK tokens will be 0s. This is different from the NLP project, where the UNK tokens are the average of the matrix.

In [25]:
emb_size = 200

def build_matrix(word_index, path):
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, emb_size))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
            if candidate in embedding_index:
                embedding_matrix[i] = embedding_index[candidate]
                break
    return embedding_matrix

In [26]:
# EMBEDDING_FILES = ['/Users/emiljoswin/study/playground/jupyter_notebooks/NLP/crawl-300d-2M.gensim', 
#                   '/Users/emiljoswin/study/playground/jupyter_notebooks/NLP/glove.840B.300d.gensim']

EMBEDDING_FILES = ['/kaggle/input/gensim-word-embeddings/glove.twitter.27B.200d.gensim']

# EMBEDDING_FILES = ['../../NLP/pre-trained-weights/gensim-embeddings-dataset/glove.twitter.27B.200d.gensim']

embedding_index = KeyedVectors.load(EMBEDDING_FILES[0], mmap='r')
# embedding_index = KeyedVectors.load(EMBEDDING_FILES[1], mmap='r')

In [27]:
m1 = build_matrix(tokenizer.word_index, EMBEDDING_FILES[0])
# m2 = build_matrix(tokenizer.word_index, EMBEDDING_FILES[1])
# m1.shape, m2.shape, 
m1.shape

(100499, 200)

In [28]:
# embedding_matrix = np.concatenate([m1, m2], axis=-1)
# embedding_matrix.shape

embedding_matrix = m1
embedding_matrix.shape

(100499, 200)

# Train Test Split

In [102]:
X = concatenate([question_body, question_title, category, answer], 1)
print(X.shape)
Y = target.to_numpy()
print(Y.shape)

test_original = concatenate([question_body_test, question_title_test, category_test, answer_test], 1)
print(test_original.shape)

(6079, 980)
(6079, 30)
(476, 980)


In [89]:
total = 6079
val_size = 700

p = np.random.permutation(list(range(total)))

val_idx = p[:val_size]
train_idx = p[val_size:]

train_x = tf.gather(X, train_idx)
train_y = tf.gather(Y, train_idx)

val_x = tf.gather(X, val_idx)
val_y = tf.gather(Y, val_idx)

print(train_x.shape, train_y.shape, val_x.shape, val_y.shape)


(5379, 980) (5379, 30) (700, 980) (700, 30)


In [90]:
print(val_idx)

[5738 2628 4041 3105  113 1266 2717 4256 4599 3825 2020  357 4728 5065
 4583 5393 4743 2328 2363 1078 4845 3708 4059 3890  205 5384  160 4077
 3074 2333 4647 6076 1914 3828  602 5914  839 1246 4060  999 5159 3412
 2215 1709 1277 3585  369 4865 1427 4809 4390 5218 4910 1950 3421  673
 1953 2168 4050 2230  293 3006 3817 5421 2442 3303 5578 4997 2391 3430
 5243 3776 2154 1517 1178 4451 2309 2103 1474 1650 5440 2684 3703 1899
 4097 4527 4963  218 4203 3294  112 1757 5889 4856 4630 5834 5184 5902
 3496 4370 3820  647 2980 4498 1208 5056 3079   49 4068  284 5318 1652
 2531 2254 1039 5681 5934 4300 4189 5663 1661 4192 5167 5612 5927 3612
 3249 5383 5524 4986   15 3002  967  624  247 1570  200  135 4334 5217
 1282 2222 5842 1225 4876 4895 3508  644 4628 2424 2337 5183   52    5
  630 3730 2344 4170 4099 4941 2566 5207 4112 1692 3851 4756  747  149
  907 5189  628 4979 4749 1342 3929 5106 3532 5066  152 6024 4205  772
 3654 1784 4863 1059 3220  450 1399 2967 1365 3517 4100 1941 2960 5246
 2807 

In [91]:
val_x = tf.convert_to_tensor(val_x)
val_y = tf.convert_to_tensor(val_y)

train_x = tf.convert_to_tensor(train_x)
train_y = tf.convert_to_tensor(train_y)

In [92]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

(TensorShape([5379, 980]),
 TensorShape([5379, 30]),
 TensorShape([700, 980]),
 TensorShape([700, 30]))

# Evaluate

In [93]:
from scipy.stats import spearmanr
import math

def calculate_mean_spearman_co(a, b):
    """
        Calculates spearmans correlation coefficient 
        https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient
        
        In the scipy implementation if you pass x and y and if one of them is composed of all same values,
        then it returns nan. I am changing those nans to zeros
    """
    # print(a.shape, b.shape, len(a.shape))
    # print(a, b)
    if len(a.shape) == 1:
        return 0

    if a.shape[0] is None and a.shape[1] is None:
         return 0
         
    ans = [spearmanr(a[:, i], b[:, i])[0] for i in range(a.shape[1])]

    ans = [0. if math.isnan(elem) else elem for elem in ans]
    return np.mean(ans), ans

# Extras

In [51]:
# LSTM_UNITS = 64
# DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
# BATCH_SIZE = 128
# EPOCHS = 10

In [52]:
# words = Input(shape=(None,))

In [53]:
# x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
# x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
# x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

# hidden = concatenate([
#         GlobalMaxPooling1D()(x),
#         GlobalAveragePooling1D()(x),
#     ])

# hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
# hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])

In [54]:
# result = Dense(30, activation='sigmoid')(hidden)

In [55]:
# model2 = Model(inputs=words, outputs=[result])
# model2.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae'])

In [40]:
# model2.summary()

In [57]:
# model2.fit(
#             train_x, train_y,
#             batch_size=256,
#             epochs=20,
#             verbose=1,
#         )

Train on 5379 samples
Epoch 1/20
5379/5379 [==============================] - 11s 2ms/sample - loss: 0.0673 - mae: 0.1898
Epoch 2/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0577 - mae: 0.1674
Epoch 3/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0559 - mae: 0.1630
Epoch 4/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0545 - mae: 0.1608
Epoch 5/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0540 - mae: 0.1589
Epoch 6/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0521 - mae: 0.1560
Epoch 7/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0524 - mae: 0.1554
Epoch 8/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0514 - mae: 0.1538
Epoch 9/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0502 - mae: 0.1514
Epoch 10/20
5379/5379 [==============================] - 7s 1ms/sample - loss: 0.0499 

In [58]:
# val_predictions = model2.predict(val_x)
# m, _ = calculate_mean_spearman_co(val_predictions, val_y)
# print("val_spearman", m)

val_spearman 0.2998429770872825


# Model as a class

In [94]:
LSTM_UNITS = 64
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
BATCH_SIZE = 128
EPOCHS = 10
OUTPUT_CLASSES = 30

In [95]:
class MyModel(models.Model):
    def __init__(self, embedding_matrix):
        super(MyModel, self).__init__()
        self.emb = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)
        self.b1 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))
        self.b2 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))
#         self.attn_weight = tf.Variable(tf.random.normal((LSTM_UNITS*2, 1)))
        self.g1 = GlobalMaxPooling1D()
        self.g2 = GlobalAveragePooling1D()
        
        self.d1 = Dense(DENSE_HIDDEN_UNITS, activation='relu')
        self.d2 = Dense(DENSE_HIDDEN_UNITS, activation='relu')
        self.linear = Dense(OUTPUT_CLASSES, activation='sigmoid')
        
    def attn(self, inputs):
        """
            inputs: [bsz x 980 x LSTM_UNITS*2]
            output: [bsz x LSTM_UNITS]
        """
        
        # (1 x 128) (bsz x seq x 128 )T  => (2 x seq)
        check = tf.einsum('ij,kjm->kim',
                          tf.transpose(self.attn_weight), 
                          tf.transpose(inputs, [0, 2, 1])) # (bsz x 1 x 980) - a score for each time step
        
        check = tf.squeeze(check) # (bsz x 980)
        check = tf.nn.softmax(check, axis=1) # attention weight (bsz x 980)
        
        #(bsz x seq x 128)T (bsz x seq) => (bsz x 128)
        check = tf.einsum('ijk,ki->ij', 
                          tf.transpose(inputs, [0, 2, 1]), 
                          tf.transpose(check) ) # (bsz x LSTM_UNITS) - best of each 980 time-steps
        
        return check

        
        
    def call(self, inputs):
        x = self.emb(inputs)
        x = self.b1(x)
        x = self.b2(x)
#         x1 = self.attn(x) # (bsz x LSTM_UNITS*2)
#         x2 = self.attn(x)
        x = concatenate([self.g1(x), self.g2(x)])
        
        x = add([x, self.d1(x)])
        x = add([x, self.d2(x)])
        x = self.linear(x)
        return x

In [96]:
model3 = MyModel(embedding_matrix)

In [97]:
# out = model3(train_x[0:2, :])

In [98]:
# out.shape

In [99]:
model3.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae'])

In [100]:
# NOTE: READ THIS
# ```````````````
# This is not real cross-validation. I am actually trying to see at what epoch number is the validation 
# Spearman score starting to decrease. So, k here is just the iteration number.
# I chose this method because each time, I will be validating it on a different random sample. A fixed sample
# is not picked because, I am not able to come up with a good validation dataset.

def train(model, k, X, Y, bsz=256, epochs=10):
    if k < 2:
        k = 2
    print(f'Doing {k}-Fold CV, bsz: {bsz}, epochs: {epochs}')
    kf = KFold(n_splits=k)
    
    train_scores = []
    val_scores = []
    for i, (train_idx, val_idx) in enumerate(kf.split(X)):
        train_x = tf.gather(X, train_idx)
        train_y = tf.gather(Y, train_idx)

        val_x = tf.gather(X, val_idx)
        val_y = tf.gather(Y, val_idx)
        
        print(f'Running the fold: {i+1}')
        model.fit(
            train_x, train_y,
            batch_size=256,
            epochs=20,
            verbose=1,
        )
        
        train_predictions = model.predict(train_x)
        m1, _ = calculate_mean_spearman_co(train_predictions, train_y)        
        
        val_predictions = model.predict(val_x)
        m2, _ = calculate_mean_spearman_co(val_predictions, val_y)
        
        train_scores.append(m1)
        val_scores.append(m2)
        print(f'Train score: {m2}')      
        print(f'Validation score: {m1}')  
        return train_scores, val_scores

In [81]:
# train(model3, 10, X, Y)

Doing 10-Fold CV
Running the fold: 0
Train on 5471 samples
Epoch 1/20
5471/5471 [==============================] - 14s 2ms/sample - loss: 0.0703 - mae: 0.1960
Epoch 2/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0571 - mae: 0.1675
Epoch 3/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0549 - mae: 0.1624
Epoch 4/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0536 - mae: 0.1592
Epoch 5/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0528 - mae: 0.1572
Epoch 6/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0520 - mae: 0.1555
Epoch 7/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0510 - mae: 0.1534
Epoch 8/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0499 - mae: 0.1516
Epoch 9/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0494 - mae: 0.1502
Epoch 10/20
5471/5471 [==========================

5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0207 - mae: 0.0920


/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Validation score: 0.45867898897999837
Running the fold: 4
Train on 5471 samples
Epoch 1/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0215 - mae: 0.0932
Epoch 2/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0215 - mae: 0.0936
Epoch 3/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0206 - mae: 0.0917
Epoch 4/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0206 - mae: 0.0916
Epoch 5/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0205 - mae: 0.0911
Epoch 6/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0198 - mae: 0.0900
Epoch 7/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0202 - mae: 0.0904
Epoch 8/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0195 - mae: 0.0892
Epoch 9/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0196 - mae: 0.0892
Epoch 10/20
5471/5471 [======

5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0112 - mae: 0.0669
Validation score: 0.5800397863628175
Running the fold: 8
Train on 5471 samples
Epoch 1/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0116 - mae: 0.0679
Epoch 2/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0115 - mae: 0.0676
Epoch 3/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0114 - mae: 0.0676
Epoch 4/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0112 - mae: 0.0666
Epoch 5/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0109 - mae: 0.0660
Epoch 6/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0111 - mae: 0.0663
Epoch 7/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0109 - mae: 0.0660
Epoch 8/20
5471/5471 [==============================] - 7s 1ms/sample - loss: 0.0108 - mae: 0.0658
Epoch 9/20
5471/5471 [===================

/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [83]:
# # Just verifying the train-prediction spearman score
# all_predictions = model3.predict(X)
# m, _ = calculate_mean_spearman_co(all_predictions, Y)
# print(m)

0.6387985168162696


In [101]:
type(X), type(Y)

(tensorflow.python.framework.ops.EagerTensor, numpy.ndarray)

In [105]:
# After computing the total epoch number
model3.fit(
            X, tf.convert_to_tensor(Y),
            batch_size=256,
            epochs=100,
            verbose=1,
        )

Train on 6079 samples
Epoch 1/100
6079/6079 [==============================] - 15s 2ms/sample - loss: 0.0662 - mae: 0.1873
Epoch 2/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0561 - mae: 0.1651
Epoch 3/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0543 - mae: 0.1608
Epoch 4/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0531 - mae: 0.1576
Epoch 5/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0520 - mae: 0.1550
Epoch 6/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0508 - mae: 0.1534
Epoch 7/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0498 - mae: 0.1512
Epoch 8/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0491 - mae: 0.1497
Epoch 9/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0482 - mae: 0.1478
Epoch 10/100
6079/6079 [==============================] - 8s 1ms/sample - los

Epoch 82/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0191 - mae: 0.0884
Epoch 83/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0188 - mae: 0.0878
Epoch 84/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0189 - mae: 0.0877
Epoch 85/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0185 - mae: 0.0869
Epoch 86/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0186 - mae: 0.0872
Epoch 87/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0184 - mae: 0.0867
Epoch 88/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0179 - mae: 0.0855
Epoch 89/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0180 - mae: 0.0858
Epoch 90/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0177 - mae: 0.0847
Epoch 91/100
6079/6079 [==============================] - 8s 1ms/sample - loss: 0.0177 - ma

In [ ]:
# val_predictions = model3.predict(val_x)
# m, _ = calculate_mean_spearman_co(val_predictions, val_y)
# print("val_spearman", m) #0.3237 => gave 0.289

# Saving the predictions

In [ ]:
predictions = model3.predict(test_original)
print(predictions.shape)

In [ ]:
target_cols = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [ ]:
loc = '/kaggle/input/google-quest-challenge/sample_submission.csv'
# loc = 'google-quest-challenge/sample_submission.csv'
sub = pd.read_csv(loc)

for col_index, col in enumerate(target_cols):
    sub[col] = predictions[:, col_index]

In [ ]:
sub.to_csv('submission.csv', index = False)